In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import csv
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
driver = webdriver.Chrome()
driver.get("https://ncrdb.usga.org/NCRDB/")

In [3]:
filename = "../csv_files/course_id_directory.csv"
  
# initializing the titles and rows list 
rows = [] 
  
# reading csv file 
with open(filename, 'r') as csvfile: 
    # creating a csv reader object 
    csvreader = csv.reader(csvfile) 
      
    # extracting field names through first row 
    # fields = csvreader.next() 
  
    # extracting each data row one by one 
    for row in csvreader: 
        rows.append(row) 
  
    # get total number of rows 
    print("Total no. of rows: %d"%(csvreader.line_num)) 

Total no. of rows: 16534


In [6]:
all_courses = pd.DataFrame()

for i in range(16422,len(rows)):
    course_id = rows[i][0]
    driver.get(f"http://ncrdb.usga.org/courseTeeInfo.aspx?CourseID={course_id}")
    time.sleep(2)
    course_name = driver.find_element_by_xpath('//*[@id="gvCourseTees"]/tbody/tr[2]/td[1]').text
    city = driver.find_element_by_xpath('//*[@id="gvCourseTees"]/tbody/tr[2]/td[2]').text
    state = driver.find_element_by_xpath('//*[@id="gvCourseTees"]/tbody/tr[2]/td[3]').text
    print(i,course_name,city,state)

    try:
        course_tees = driver.find_element_by_id("gvTee").get_attribute('outerHTML')
        df  = pd.read_html(course_tees)
    except:
        print("No Course Ratings found")
        df = list()
        df.append(pd.DataFrame({'Tee Name':['NR'],'Par':['NR'],'Course Rating (18)':['NR'],'Slope Rating (18)':['NR'],'Front (9)':['NR'],'Back (9)':['NR'],'Bogey Rating (18)':['NR'],'Gender':['NR']}))
    
    df[0]['Course Name'] = course_name
    df[0]['Course ID'] = course_id
    df[0]['City'] = city
    df[0]['State']= state
    all_courses = all_courses.append(df[0])

all_courses = all_courses[['Course Name','Course ID','City','State','Tee Name','Par','USGA Course Rating (18)','Slope Rating (18)','Front (9)','Back (9)','Bogey Rating (18)','Gender']].reset_index(drop=True)
all_courses.head()

16422 Wind River Golf Course - Wind River Golf Course Riverton WY
16423 ALCESTER GC - ALCESTER GC ALCESTER SD
16424 Arrowhead Country Club - Arrowhead Country Club Rapid City SD
16425 Bakker Crossing Golf Course - Bakker Crossing Golf Course Sioux Falls SD
16426 BELLE FOURCHE CC - BELLE FOURCHE CC BELLE FOURCHE SD
16427 Bison Country Club - Bison Country Club Bison SD
16428 BON HOMME CC - BON HOMME CC TYNDALL SD
16429 BOULDER CANYON CC - BOULDER CANYON CC STURGIS SD
16430 BOWDLE GC - BOWDLE GC BOWDLE SD
16431 Brandon Golf Course - Brandon Golf Course Brandon SD
16432 BRITTON CC - BRITTON CC BRITTON SD
16433 BROADLAND CREEK NATIONAL GC - BROADLAND CREEK NATIONAL GC HURON SD
16434 BROOKINGS COUNTRY CLUB - BROOKINGS COUNTRY CLUB BROOKINGS SD
16435 Burke Golf Club - Burke Golf Club Burke SD
16436 Castlewood Golf Course - Castlewood Golf Course Castlewood SD
16437 Cattail Crossing GC - Blue/Yellow Watertown SD
16438 Cattail Crossing GC - Red/Blue Watertown SD
16439 Cattail Crossing GC - Yel

/Users/rossnicholson/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


16530 Whetstone Creek Golf Course - Whetstone Creek Golf Course Milbank SD
16531 Wild Oak Golf Course - Wild Oak Golf Course Mitchell SD
16532 Willow Run Golf Course - Willow Run Golf Course Sioux Falls SD
16533 Winner Country Club - Winner Country Club Winner SD


KeyError: "['USGA Course Rating (18)'] not in index"

In [7]:
all_courses = all_courses[['Course Name','Course ID','City','State','Tee Name','Par','Course Rating (18)','Slope Rating (18)','Front (9)','Back (9)','Bogey Rating (18)','Gender']].reset_index(drop=True)
all_courses.head()

,Course Name,Course ID,City,State,Tee Name,Par,Course Rating (18),Slope Rating (18),Front (9),Back (9),Bogey Rating (18),Gender
0,Wind River Golf Course - Wind River Golf Course,10133,Riverton,WY,Blue,36,35,122,35.0 / 122,/,46.3,M
1,Wind River Golf Course - Wind River Golf Course,10133,Riverton,WY,White,36,34.1,117,34.1 / 117,/,45,M
2,Wind River Golf Course - Wind River Golf Course,10133,Riverton,WY,Red,36,34.6,117,34.6 / 117,/,48.4,F
3,ALCESTER GC - ALCESTER GC,6017,ALCESTER,SD,Blue,0,32.3,109,32.3 / 109,/,42.4,M
4,ALCESTER GC - ALCESTER GC,6017,ALCESTER,SD,White,0,31.5,105,31.5 / 105,/,41.3,M


In [8]:
all_courses.to_csv('temp_course_rating_db.csv',index=False)